In [1]:
%pip install torch==2.3.0+cu121
%pip install roboflow
%pip install clearml

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.4  Python-3.12.2 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Setup complete  (12 CPUs, 15.7 GB RAM, 386.7/465.7 GB disk)


In [3]:
from roboflow import Roboflow
from ultralytics import YOLO
from clearml import Task

In [4]:
import torch
print(f'PyTorch version: {torch.__version__}')
print('*'*10)
print(f'_CUDA version: ')
!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.3.0+cu121
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Feb__8_05:53:42_Coordinated_Universal_Time_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0
**********
CUDNN version: 8801
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 4050 Laptop GPU


In [5]:
# YOLO model version

model_version = 'yolov8m'

In [6]:
#task.close()

task = Task.init(project_name="Diplom", task_name=f"{model_version}_task")

ClearML Task: overwriting (reusing) task id=b6479f117ec944fb99caa9275a7e55e3
2024-05-09 22:05:52,933 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2024-05-09 22:05:52,964 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/6e8b14edcdd84c289b72a5c8292790e2/experiments/b6479f117ec944fb99caa9275a7e55e3/output/log


ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [7]:
print(torch.__version__)
print(torch.cuda.is_available())

2.3.0+cu121
True


In [8]:
## Check for CUDA device and set it
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
#print(f'Using device: {device}')

# Load model
model = YOLO(f"{model_version}.pt") 
#model.to(device)

2024-05-09 22:06:01,574 - clearml.model - INFO - Selected model id: 53e52ffa76b348f5a0cd67aebe0acc3e


In [9]:
dataset = 'C:/Users/greyb/OneDrive/Desktop/Potok/Study/Diplom/data/data.yaml'

In [10]:
'''
# Training.
results = model.train(
   data=dataset,
   imgsz=1024,
   epochs=10,
   batch=4,
   device=0,
   seed=1,
   cls=3.0,
   dfl=1.8,
   box=6.5,
   dropout=0.0008,
   name=f'{model_version}')
'''

"\n# Training.\nresults = model.train(\n   data=dataset,\n   imgsz=1024,\n   epochs=10,\n   batch=4,\n   device=0,\n   seed=1,\n   cls=3.0,\n   dfl=1.8,\n   box=6.5,\n   dropout=0.0008,\n   name=f'{model_version}')\n"

In [11]:
best_model_path = 'C:/Users/greyb/OneDrive/Desktop/Potok/Study/Diplom/network/runs/detect/yolov8m5_BEST_V2/weights/best.pt'


In [12]:
best_model = YOLO(best_model_path)

2024-05-09 22:06:06,629 - clearml.model - INFO - Selected model id: 823f5b1b0c454c54a60b3e96df1d6631


In [13]:
import yaml

import sys
sys.path.insert(0, '../toolkit')

import PreprocessData as prepr_data
from utils import load_symbol_map, cls_to_symbol, sort_coordinates_by_row, input_spaces, to_rus_symbols

import os
import cv2

In [14]:
path_to_symbol_map = 'C:/Users/greyb/OneDrive/Desktop/Potok/Study/Diplom/data/data.yaml'

symbol_map = load_symbol_map(path_to_symbol_map, 'names')

In [35]:
test_img_path = '../test/test/testIMG_2.jpg' # test_image

img = cv2.imread(test_img_path, cv2.IMREAD_UNCHANGED)

In [36]:
img_size = 1024
blur_value = 0
exposure_values = [0.45, 50] # [0.45, 50] 

preprocessing_image = prepr_data.Preprocessing(img_size, blur_value, exposure_values).preprocess_image(img)

prepr_img_path = '../test/test/prepr_imgs'
cv2.imwrite(os.path.join(prepr_img_path, 'test.jpg'), preprocessing_image)

True

In [17]:
'''
cv2.imshow("Preprocessed image", preprocessing_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

'\ncv2.imshow("Preprocessed image", preprocessing_image)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'

In [41]:
#best_model.predict(test_img_path)

results = best_model.predict(f'{prepr_img_path}/test.jpg', augment=True, save_crop=False, show_conf=False, line_width=1, save=True, agnostic_nms=True)


image 1/1 c:\Users\greyb\OneDrive\Desktop\Potok\Study\Diplom\network\..\test\test\prepr_imgs\test.jpg: 1024x1024 6 as, 2 bs, 1 c, 2 chs, 3 ds, 5 es, 8 is, 1 iKratkoe, 2 iis, 3 ks, 3 ls, 6 _ns, 5 os, 2 ps, 5 rs, 1 s, 4 ts, 1 v, 3 xs, 2 _ys, 5 yas, 1 z, 1 zapyataya, 357.6ms
Speed: 10.7ms preprocess, 357.6ms inference, 4.1ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to runs\detect\predict18


In [42]:
# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    #masks = result.masks  # Masks object for segmentation masks outputs
    #keypoints = result.keypoints  # Keypoints object for pose outputs
    #probs = result.probs  # Probs object for classification outputs
    #obb = result.obb  # Oriented boxes object for OBB outputs
    #result.show()  # display to screen
    #result.save(filename='result.jpg')  # save to disk

In [43]:
print(boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([42., 22., 52., 26., 46., 42., 30., 26., 50., 52., 33., 46., 30., 30., 30., 33., 51., 43., 36., 46., 39., 22., 30., 52., 23., 26., 50., 19., 23., 36., 40., 40., 40., 41., 20., 42., 26., 51., 42., 41., 21., 46., 30., 39., 40., 36., 30., 39., 54., 39., 23., 19., 42., 40., 20., 32., 52., 50., 52., 19., 30., 19.,
        39., 19., 26., 31., 49., 33., 39., 32., 19., 55.], device='cuda:0')
conf: tensor([0.9590, 0.9585, 0.9506, 0.9425, 0.9366, 0.9337, 0.9321, 0.9293, 0.9282, 0.9269, 0.9259, 0.9258, 0.9244, 0.9241, 0.9238, 0.9235, 0.9230, 0.9229, 0.9211, 0.9203, 0.9197, 0.9170, 0.9164, 0.9162, 0.9160, 0.9156, 0.9138, 0.9137, 0.9127, 0.9097, 0.9091, 0.9085, 0.9076, 0.9068, 0.9045, 0.9022, 0.9013, 0.8991, 0.8983,
        0.8978, 0.8951, 0.8948, 0.8948, 0.8943, 0.8943, 0.8908, 0.8881, 0.8871, 0.8870, 0.8869, 0.8780, 0.8771, 0.8692, 0.8683, 0.8679, 0.8671, 0.8628, 0.8624, 0.8602, 0.8592, 0.8568, 0.8496, 0.8436, 0.8424, 0.8415, 0

In [44]:
result_classes = cls_to_symbol(boxes.cls, symbol_map)
#result_classes

In [45]:
sorted_result = sort_coordinates_by_row(boxes.xywhn, result_classes, threshold=0.01)
print(len(sorted_result))
#sorted_result

3


In [46]:
sorted_result_1DIM = []
for sublist in sorted_result:
    sorted_result_1DIM.extend(sublist)

In [47]:
sorted_result_1DIM_with_spaces = input_spaces(sorted_result_1DIM, threshold=0.02)
sorted_result_1DIM_with_spaces

[[0.36187732219696045, 0.5334787368774414, ('_y', 51)],
 [0.3746684193611145, 0.5336769819259644, ('ch', 22)],
 [0.38777467608451843, 0.5338556170463562, ('e', 26)],
 [0.40062808990478516, 0.5339553356170654, ('b', 20)],
 [0.41385143995285034, 0.5328406691551208, ('_n', 39)],
 [0.42655813694000244, 0.5326462388038635, ('a', 19)],
 [0.43942588567733765, 0.5330103635787964, ('ya', 52)],
 [0.4576647877693176, 0.5327951908111572, ('a', 19)],
 [0.4700397253036499, 0.5328893661499023, ('_y', 51)],
 [0.4820702075958252, 0.5334415435791016, ('d', 23)],
 [0.4947817921638489, 0.5336084365844727, ('i', 30)],
 [0.5070950984954834, 0.5338000059127808, ('t', 46)],
 [0.5190868377685547, 0.5338377952575684, ('o', 40)],
 [0.53148353099823, 0.5337958335876465, ('r', 42)],
 [0.5437362790107727, 0.533916175365448, ('i', 30)],
 [0.5563647747039795, 0.5330532193183899, ('ya', 52)],
 [0.5743675827980042, 0.5332986116409302, ('d', 23)],
 [0.5868257284164429, 0.5335069894790649, ('l', 36)],
 [0.598504304885864

In [25]:
res_rus_text_list = to_rus_symbols(sorted_result_1DIM_with_spaces)

In [26]:
result_text_str = ''.join(res_rus_text_list)
result_text_str

'учебнаяаудиториядляпроведения лабораторных,практическихи лекционныхзанятий'

In [27]:
from pyaspeller import YandexSpeller
from autocorrect import Speller

In [28]:
spell = Speller()
speller = YandexSpeller()

In [29]:
result_text_str = spell(result_text_str)
result_text_str = speller.spelled(result_text_str)
result_text_str

'учебная аудитория для проведения лабораторных,практических и лекционных занятий'

In [50]:
import pyttsx3

engine = pyttsx3.init()
engine.save_to_file(result_text_str , 'test2.wav')

engine.say(result_text_str)
engine.runAndWait()

In [31]:
'''
cv2.imshow("Preprocessed image", preprocessing_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

'\ncv2.imshow("Preprocessed image", preprocessing_image)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'

In [32]:
'''
coords = (0, 0, 100, 100)

x_1 = coords[0]
y_1 = coords[1]
x_2 = coords[2]
y_2 = coords[3]

start_point = (int(x_1), int(y_1))
end_point = (int(x_2), int(y_2))

image = cv2.rectangle(preprocessing_image, start_point, end_point, (0,255,0), 1)

cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

'\ncoords = (0, 0, 100, 100)\n\nx_1 = coords[0]\ny_1 = coords[1]\nx_2 = coords[2]\ny_2 = coords[3]\n\nstart_point = (int(x_1), int(y_1))\nend_point = (int(x_2), int(y_2))\n\nimage = cv2.rectangle(preprocessing_image, start_point, end_point, (0,255,0), 1)\n\ncv2.imshow("Image", image)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'